-User-Based Collaborative Filtering .Kullanıcılar arasındaki benzerlik skorlarını bulmaya çalışır. .Her bir kullanıcının satın aldığı izlediği/dinlediği materyali kullanıcı bazlı olarak tutar ve bir matrise çevirir. .X kullanıcısının henüz izlemediği ancak benzer kullanıcıların çoğunun izlediği bir film varsa bunu X kullanıcısına önerir.

-Item-Based Collaborative Filtering .Kullanıcı yerine film/müzik/satılan ürün/makale arasında ilişki tutuyor. .İzleyip bitirdiğimiz filme en yakın filme göre öneri veriyor.

Kullanacağımız kütüphaneler seaborn ,pandas ,numpy, difflib-> Kullanıcıdan film ismi girmesini isteyeceğiz o filme benzer filmleri bize getirecek. sklearn TfidfVectorizer metinleri sayısala çevirme

Yapacağım film öneri sistemi içerik ve popülerlik tabanlı bir sistem olacak bu yüzden tüm sutunları kullanmayacağım.

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#csv dosyasını aktardık ve ilk 5 satırı gösterdik.
#sütunlarımız filmin index numarası filmin bütçesini, türünü , izleyebilceğimiz sayfayı, film için özel id'sini
#filmde neler olduğunu belirten anahtar kelimeleri,orijinal dilini, ismini, konusu, popülerlik puanını
#filmi yapan prodüksiyon şirketleri,ülkeleri,yayınlanma tarihi, geliri, filmin çalışılma süresi, filmde konuşulan dillerin
#Yayınlanıp yayınlanmama durumunu, filmin sloganını, oy sayımızı ve kaç kelime olduğunu,filmin oyuncu kadrosunu,
#film ekibinde yer alanlar ve filmin yönetmenini görmekteyiz.

film_verileri = pd.read_csv("/content/drive/MyDrive/movies.csv")
film_verileri.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#veri setimizin kaç satır ve kaç sütundan olduğunu görüyoruz. Her satır bir filmi temsili ediyor
film_verileri.shape

(4803, 24)

In [ ]:
#verilerimiz hakkında bilgi aldık. 
#genres,homepage,keywords, original_language , original_title ,overview      
#production_companies,production_countries,release_date ,spoken_languages 
#status,tagline ,title,cast,crew,director -> KATEGORİK VERİLERDİR.

#index,budget,id,popularity,runtime,revenue,vote_average,vote_count-> SAYISAL VERİLERDİR..


film_verileri.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [ ]:
# veri değişkenlerimi görüntüledik.
film_verileri.dtypes 

index                     int64
budget                    int64
genres                   object
homepage                 object
id                        int64
keywords                 object
original_language        object
original_title           object
overview                 object
popularity              float64
production_companies     object
production_countries     object
release_date             object
revenue                   int64
runtime                 float64
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count                int64
cast                     object
crew                     object
director                 object
dtype: object

In [ ]:
film_verileri.isnull().values.any() #veri setimizin içinde boşluk var mı kontrol ediyor

True

In [ ]:
film_verileri.isnull().sum() #her değişken için kaç satırda boşluk olduğunu gösteriyor.

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [ ]:
#sayısal değerlerin ortalamasını,minimum,maksimum,standart sapmasını, sayısını bulduk.
film_verileri.describe().T

,count,mean,std,min,25%,50%,75%,max
index,4803.0,2.401000e+03,1.386651e+03,0.0,1200.50000,2.401000e+03,3.601500e+03,4.802000e+03
budget,4803.0,2.904504e+07,4.072239e+07,0.0,790000.00000,1.500000e+07,4.000000e+07,3.800000e+08
id,4803.0,5.716548e+04,8.869461e+04,5.0,9014.50000,1.462900e+04,5.861050e+04,4.594880e+05
popularity,4803.0,2.149230e+01,3.181665e+01,0.0,4.66807,1.292159e+01,2.831350e+01,8.755813e+02
revenue,4803.0,8.226064e+07,1.628571e+08,0.0,0.00000,1.917000e+07,9.291719e+07,2.787965e+09
runtime,4801.0,1.068759e+02,2.261193e+01,0.0,94.00000,1.030000e+02,1.180000e+02,3.380000e+02
vote_average,4803.0,6.092172e+00,1.194612e+00,0.0,5.60000,6.200000e+00,6.800000e+00,1.000000e+01
vote_count,4803.0,6.902180e+02,1.234586e+03,0.0,54.00000,2.350000e+02,7.370000e+02,1.375200e+04


In [ ]:
secilen_sutunlar = ['genres','keywords','tagline','cast','director']
secilen_sutunlar

['genres', 'keywords', 'tagline', 'cast', 'director']

In [ ]:
#Seçtiğimiz sütunlardaki boş olan değerleri boş bir alanla değiştirdik
for sutun in secilen_sutunlar:
  film_verileri[sutun] = film_verileri[sutun].fillna('')

In [ ]:
#Seçilen sütunları birleştirdik
birlesen_sutunlar = film_verileri['genres']+' '+film_verileri['keywords']+' '+film_verileri['tagline']+' '+film_verileri['cast']+' '+film_verileri['director']
birlesen_sutunlar

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [ ]:
#TfidfVectorizer kütüphanesi sayesiniden metin olan değerleri sayısala çeviriyoruz.


vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(birlesen_sutunlar)
feature_vectors


<4803x17318 sparse matrix of type '<class 'numpy.float64'>'
	with 124266 stored elements in Compressed Sparse Row format>

In [ ]:
#kosinüs benzerliğini kullanarak benzerlik puanlarını alıyoruz.
benzerlik = cosine_similarity(feature_vectors)
benzerlik

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]])

In [ ]:
#Mesela listemizin başındaki Avatar filmini seçelim bu veri seti içindeki tüm filmlerle karşılaştırmaya çalışacak 
# ve benzerlik puanı vermeye çalışacak ve ardından bu diğer tüm filmler içinde yapılacak
benzerlik.shape

(4803, 4803)

Kullanıcıdan film adını alma

In [ ]:
film_ad = input(' Favori filminizin adını giriniz: ')

 Favori filminizin adını giriniz: The Avengers


In [ ]:
#veri kümesinde verilen tüm film adlarıyla bir liste oluşturduk
liistedeki_tüm_filmler = film_verileri['title'].tolist()
liistedeki_tüm_filmler

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [ ]:
#kullanıcı tarafından verilen film adı için yakın eşleşmeyi bulduk
yakin_eslesmeyi_bul = difflib.get_close_matches(film_ad, liistedeki_tüm_filmler)
yakin_eslesmeyi_bul

['The Avengers', 'The Toxic Avenger', 'The Lovers']

In [ ]:
yakin_eslesme = yakin_eslesmeyi_bul[0]
yakin_eslesme

'The Avengers'

In [ ]:
#Filmin indeksini bulduk.
film_indeksi = film_verileri[film_verileri.title == yakin_eslesme]['index'].values[0]
film_indeksi

16

In [ ]:
#benzer filmlerin bir listesini aldık.
benzerlik_puani = list(enumerate(benzerlik[film_indeksi]))
benzerlik_puani

[(0, 0.03426340578061641),
 (1, 0.018757441743698592),
 (2, 0.03907993864609115),
 (3, 0.006602224357539788),
 (4, 0.05721539202804143),
 (5, 0.014194201691251133),
 (6, 0.02358797178187124),
 (7, 0.7860896884964055),
 (8, 0.008045023221038158),
 (9, 0.16210160555183875),
 (10, 0.07691564861356003),
 (11, 0.01217213686353538),
 (12, 0.013990444052447457),
 (13, 0.012631427434045384),
 (14, 0.1837336996264891),
 (15, 0.04563629684495581),
 (16, 0.9999999999999999),
 (17, 0.013383483902995971),
 (18, 0.02529414696558761),
 (19, 0.01188236224021718),
 (20, 0.11215733913329524),
 (21, 0.027050077660405823),
 (22, 0.007034788718083841),
 (23, 0.036653962075193565),
 (24, 0.01292652624563084),
 (25, 0.010727286350311038),
 (26, 0.4046969407600009),
 (27, 0.03121225816313765),
 (28, 0.06887119248461429),
 (29, 0.025483413379835257),
 (30, 0.11350883208096145),
 (31, 0.21487810271500662),
 (32, 0.029868095335032138),
 (33, 0.2505154038738609),
 (34, 0.0),
 (35, 0.036079460398325214),
 (36, 0.0

In [ ]:
#filmleri benzerlik puanlarına göre sıraladık. x[1] benzerlik puanı
benzer_filmleri_sirala = sorted(benzerlik_puani, key = lambda x:x[1], reverse = True) 
benzer_filmleri_sirala

[(16, 0.9999999999999999),
 (7, 0.7860896884964055),
 (85, 0.4301532854607663),
 (26, 0.4046969407600009),
 (79, 0.3933653363061258),
 (126, 0.312803363203424),
 (511, 0.2892396085894662),
 (174, 0.28862963963238886),
 (64, 0.27464767891998554),
 (182, 0.27276411277152907),
 (129, 0.26928516313639983),
 (203, 0.25660973492915823),
 (33, 0.2505154038738609),
 (788, 0.24366990820486284),
 (46, 0.24006691472148445),
 (169, 0.23702484619175357),
 (38, 0.2338952010106937),
 (4759, 0.23084821219774054),
 (68, 0.22704403782296803),
 (31, 0.21487810271500662),
 (14, 0.1837336996264891),
 (782, 0.1825529075668662),
 (870, 0.1821588304668644),
 (101, 0.18018869067098506),
 (94, 0.16311290114551133),
 (9, 0.16210160555183875),
 (1294, 0.15229231204591692),
 (1192, 0.1517599526469107),
 (241, 0.15100480467438165),
 (4401, 0.1480526961044298),
 (2197, 0.14650796705575336),
 (1720, 0.14569980465005258),
 (713, 0.1452783854259603),
 (1740, 0.14444382118424323),
 (122, 0.14409292201469673),
 (242, 0.1

In [ ]:
#index numarasına göre benzer filmlerin isimlerini yazdırdık.
print('Senin için önerilenler : \n')

i = 1

for film in benzer_filmleri_sirala:
  index = film[0]
  indekse_gore_filmler = film_verileri[film_verileri.index==index]['title'].values[0]
  if (i<50):
    print(i, '.',indekse_gore_filmler)
    i+=1

Senin için önerilenler : 

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Captain America: The Winter Soldier
4 . Captain America: Civil War
5 . Iron Man 2
6 . Thor: The Dark World
7 . X-Men
8 . The Incredible Hulk
9 . X-Men: Apocalypse
10 . Ant-Man
11 . Thor
12 . X2
13 . X-Men: The Last Stand
14 . Deadpool
15 . X-Men: Days of Future Past
16 . Captain America: The First Avenger
17 . The Amazing Spider-Man 2
18 . The Image Revolution
19 . Iron Man
20 . Iron Man 3
21 . Man of Steel
22 . The Spirit
23 . Superman II
24 . X-Men: First Class
25 . Guardians of the Galaxy
26 . Batman v Superman: Dawn of Justice
27 . Serenity
28 . Spawn
29 . Teenage Mutant Ninja Turtles: Out of the Shadows
30 . The Helix... Loaded
31 . What's Your Number?
32 . Kick-Ass
33 . The Horse Whisperer
34 . Kick-Ass 2
35 . X-Men Origins: Wolverine
36 . Fantastic Four
37 . 2 Guns
38 . Zodiac
39 . Red Sonja
40 . TMNT
41 . Street Fighter: The Legend of Chun-Li
42 . Watchmen
43 . Sin City
44 . Money Train
45 . A Scanner D

In [ ]:
film_ad = input(' Favori filminizin adını giriniz: ')

liistedeki_tüm_filmler = film_verileri['title'].tolist()
liistedeki_tüm_filmler

yakin_eslesmeyi_bul = difflib.get_close_matches(film_ad, liistedeki_tüm_filmler)
yakin_eslesmeyi_bul

yakin_eslesme = yakin_eslesmeyi_bul[0]
yakin_eslesme

film_indeksi = film_verileri[film_verileri.title == yakin_eslesme]['index'].values[0]
film_indeksi

benzerlik_puani = list(enumerate(benzerlik[film_indeksi]))
benzerlik_puani

benzer_filmleri_sirala = sorted(benzerlik_puani, key = lambda x:x[1], reverse = True) 
benzer_filmleri_sirala

print('Senin için önerilenler : \n')

i = 1

for film in benzer_filmleri_sirala:
  index = film[0]
  indekse_gore_filmler = film_verileri[film_verileri.index==index]['title'].values[0]
  if (i<50):
    print(i, '.',indekse_gore_filmler)
    i+=1

 Favori filminizin adını giriniz: Interstaller
Senin için önerilenler : 

1 . Interstellar
2 . The Dark Knight Rises
3 . The Matrix
4 . The Martian
5 . Dear Frankie
6 . Argo
7 . The Matrix Revolutions
8 . The Matrix Reloaded
9 . The Terminator
10 . Armageddon
11 . Terminator Genisys
12 . Contact
13 . Terminator Salvation
14 . The Killer Inside Me
15 . Gandhi, My Father
16 . The Tree of Life
17 . Get Smart
18 . Back to the Future
19 . Terminator 3: Rise of the Machines
20 . The Prestige
21 . Batman Begins
22 . Dragonslayer
23 . WarGames
24 . Little Nicky
25 . Superman III
26 . The Other Side of Heaven
27 . House at the End of the Street
28 . Good Deeds
29 . Mortal Kombat: Annihilation
30 . Mamma Mia!
31 . Triple 9
32 . The Prophecy
33 . The Cave
34 . The Dark Knight
35 . True Romance
36 . The Cotton Club
37 . Capricorn One
38 . To Die For
39 . The Last Emperor
40 . Stealth
41 . The Hustler
42 . Transcendence
43 . Snitch
44 . 200 Cigarettes
45 . Rudderless
46 . The Ice Storm
47 . He Got 

0       62
1       53
2       46
4       43
3       41
        ..
648      1
5862     1
3583     1
1174     1
2078     1
Name: vote_count, Length: 1609, dtype: int64